In [2]:
import numpy as np
from emo_utils import *
from keras.layers import Input, Dense, Activation, LSTM, Bidirectional, RepeatVector, Dot, Concatenate, Embedding
from keras.layers import Conv1D
from keras.optimizers import Adam
from keras.models import Model
from keras.initializers import glorot_uniform

In [3]:
data, labels, max_length=load_data()

In [4]:
Tx=max_length
Tx

66

In [5]:
label_dict, y_hot=labels_to_one_hot(labels)

In [6]:
word_to_vec, Glove_words, word_to_index=load_GloVe()

In [7]:
X_index=data_to_index(data, word_to_index, Glove_words, Tx)

In [8]:
def pretrained_embedding_layer(word_to_vec, word_to_index):
    
    vocab_size=len(word_to_index)+1     ### +1 for unknown words which are not present in the Glove_words
    
    emb_dim= word_to_vec['this'].shape[0]
    
    
    embedding_matrix=np.zeros((vocab_size, emb_dim))
    
    for w, i in word_to_index.items():
        
        embedding_matrix[i]=word_to_vec[w]
    
    embedding_layer= Embedding(input_dim=vocab_size, output_dim=emb_dim, trainable=False)  ## can only be used as 
                                                                                            ### the first layer in a model
    embedding_layer.build((None,))
    
    embedding_layer.set_weights([embedding_matrix])
    
    return embedding_layer
    

In [61]:
repeator=RepeatVector(Tx)
concatenator=Concatenate(axis=-1)
dotor=Dot(axes=1)

In [62]:
def one_step_attention(s_prev, a):
    
    s_prev=repeator(s_prev)
    
    concat=concatenator([s_prev,a])
    
    e=Dense(units=32, activation='tanh')(concat)
    
    e=Dense(units=64, activation='relu')(e)
    
    alphas= Activation("softmax")(e)
    
    context=dotor([alphas, a])
    
    return context


In [63]:
def model(word_to_vec, word_to_index, n_a, n_s, Tx, label_dict_size):
    
    X_input=Input(shape=(Tx,))
    
    embedding_layer= pretrained_embedding_layer(word_to_vec, word_to_index)
    
    X=embedding_layer(X_input)                  ## X is the embeddings of input  
    
    a=Bidirectional(LSTM(units=n_a, return_sequences=True))(X)
    
    a=Conv1D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer=glorot_uniform(seed=0))(a)
    
    s0=Input(shape=(n_s,))
    c0=Input(shape=(n_s,))
    
    context=one_step_attention(s0, a)
    
    s,_,c= LSTM(units=n_s, return_state=True)([context, s0,c0])
    
    out= Dense(units=label_dict_size, activation='softmax')(s)
    
    model=Model(inputs=[X_input, s0, c0], outputs=out)
    
    return model
    

In [64]:
n_a=32
n_s=64
model=model(word_to_vec, word_to_index, n_a=32,n_s=64, Tx=Tx, label_dict_size=len(label_dict))

In [65]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_38 (InputLayer)            (None, 66)            0                                            
____________________________________________________________________________________________________
embedding_14 (Embedding)         (None, 66, 100)       40000100    input_38[0][0]                   
____________________________________________________________________________________________________
input_39 (InputLayer)            (None, 64)            0                                            
____________________________________________________________________________________________________
bidirectional_13 (Bidirectional) (None, 66, 64)        34048       embedding_14[0][0]               
___________________________________________________________________________________________

In [66]:
opt=Adam(lr=0.02, beta_1=0.9, beta_2=0.999, decay=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [67]:
m=len(data)
n_s=64
s0=np.zeros((m, n_s))
c0=np.zeros((m, n_s))

In [68]:
model.fit([X_index,s0,c0], y_hot, epochs=15, batch_size=128)   ### batch_size in the form 2^k

Epoch 1/15
16000/16000 [==============================] - 88s - loss: 1.5639 - acc: 0.3624    
Epoch 2/15
16000/16000 [==============================] - 81s - loss: 1.2576 - acc: 0.5307    
Epoch 3/15
16000/16000 [==============================] - 84s - loss: 1.1203 - acc: 0.5837    
Epoch 4/15
16000/16000 [==============================] - 82s - loss: 0.9270 - acc: 0.6787    
Epoch 5/15
16000/16000 [==============================] - 81s - loss: 0.7964 - acc: 0.7309    
Epoch 6/15
16000/16000 [==============================] - 81s - loss: 0.7088 - acc: 0.7631    
Epoch 7/15
16000/16000 [==============================] - 81s - loss: 0.6509 - acc: 0.7806    
Epoch 8/15
16000/16000 [==============================] - 81s - loss: 0.6102 - acc: 0.7923    
Epoch 9/15
16000/16000 [==============================] - 81s - loss: 0.5832 - acc: 0.8013    
Epoch 10/15
16000/16000 [==============================] - 81s - loss: 0.5543 - acc: 0.8111    
Epoch 11/15
16000/16000 [========================